### Tensorflow Code

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [2]:
%matplotlib inline

In [3]:
mb_size = 32
X_dim = 784
z_dim = 64
h_dim = 128
lr = 1e-3
d_steps = 3

In [4]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [6]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [7]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]
theta_D = [D_W1, D_W2, D_b1, D_b2]

In [8]:
def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    out = tf.matmul(D_h1, D_W2) + D_b2
    return out

In [9]:
G_sample = generator(z)

D_real = discriminator(X)
D_fake = discriminator(G_sample)

D_loss = 0.5 * (tf.reduce_mean((D_real - 1)**2) + tf.reduce_mean(D_fake**2))
G_loss = 0.5 * tf.reduce_mean((D_fake - 1)**2)

D_solver = (tf.train.AdamOptimizer(learning_rate=lr)
            .minimize(D_loss, var_list=theta_D))
G_solver = (tf.train.AdamOptimizer(learning_rate=lr)
            .minimize(G_loss, var_list=theta_G))

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out_lsgan/'):
    os.makedirs('out_lsgan/')

In [11]:
i = 0

for it in range(10000):
    for _ in range(d_steps):
        X_mb, _ = mnist.train.next_batch(mb_size)
        z_mb = sample_z(mb_size, z_dim)

        _, D_loss_curr = sess.run(
            [D_solver, D_loss],
            feed_dict={X: X_mb, z: z_mb}
        )

    X_mb, _ = mnist.train.next_batch(mb_size)
    z_mb = sample_z(mb_size, z_dim)

    _, G_loss_curr = sess.run(
        [G_solver, G_loss],
        feed_dict={X: X_mb, z: sample_z(mb_size, z_dim)}
    )

    if it % 1000 == 0:
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr))

        samples = sess.run(G_sample, feed_dict={z: sample_z(16, z_dim)})

        fig = plot(samples)
        plt.savefig('out_lsgan/{}.png'
                    .format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

Iter: 0; D_loss: 0.7375; G_loss: 0.7689
Iter: 1000; D_loss: 0.0006068; G_loss: 0.5007
Iter: 2000; D_loss: 0.001609; G_loss: 0.5251
Iter: 3000; D_loss: 0.004376; G_loss: 0.5207
Iter: 4000; D_loss: 0.009899; G_loss: 0.5414
Iter: 5000; D_loss: 0.01144; G_loss: 0.4331
Iter: 6000; D_loss: 0.03321; G_loss: 0.4393
Iter: 7000; D_loss: 0.05799; G_loss: 0.4265
Iter: 8000; D_loss: 0.0678; G_loss: 0.3894
Iter: 9000; D_loss: 0.05428; G_loss: 0.5191


v. ![img000](out_vanilla/000.png) l. ![img001](out_lsgan/000.png)
v. ![img001](out_vanilla/001.png) l. ![img001](out_lsgan/001.png)
v. ![img002](out_vanilla/002.png) l. ![img002](out_lsgan/002.png)
v. ![img003](out_vanilla/003.png) l. ![img003](out_lsgan/003.png)
v. ![img004](out_vanilla/004.png) l. ![img004](out_lsgan/004.png)
v. ![img005](out_vanilla/005.png) l. ![img005](out_lsgan/005.png)
v. ![img006](out_vanilla/006.png) l. ![img006](out_lsgan/006.png)
v. ![img007](out_vanilla/007.png) l. ![img007](out_lsgan/007.png)
v. ![img008](out_vanilla/008.png) l. ![img008](out_lsgan/008.png)
v. ![img009](out_vanilla/009.png) l. ![img009](out_lsgan/009.png)